In [ ]:



import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/330k-arabic-sentiment-reviews/arabic_sentiment_reviews.csv


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
import re

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
# to run more than output in the same cell
pd.set_option('display.max_colwidth', None)

In [ ]:
df = pd.read_csv(os.path.join(dirname, filename))
df.head()

,label,content
0,1,النعال المريحة: أرتدي هذه النعال كثيرًا!فهي دافئة ومريحة وبأسعار معقولة لجودة رائعة.زوجي وأنا على حد سواء لدينا زوج ونحن نحبهم!
1,1,منتج جميل ، خدمة سيئة: لقد اشتريت زوجًا من النعال الباو الدب.باتباع إرشادات وصف المنتج ، ارتفعت حجمًا للتناسب.عندما وصلوا ، كانوا لطيفين حقًا وأحبوا.كانت كبيرة جدا.حاولت إعادة ترتيب ولم تكن متوفرة.حاولت الاتصال بـ Claussette عبر الهاتف وقيل لها البريد الإلكتروني.لقد أرسلت عبر البريد الإلكتروني معضلي وانتظرت أسبوعًا ، ولم ترد ، لقد أرسلت رسالتي الإلكترونية مرة أخرى ، قبل ثلاثة أيام.لا يوجد استجابة.أريد فقط استبدال العنصر ، أو الحصول على إجابة.
2,1,جيد للأشياء الصغيرة: هذا يعمل بشكل جيد لالتقاط قطع صغيرة من المجوهرات ، ولكن الذهاب ببطء.إنه مفيد ، لكن المغناطيس ليس قويًا جدًا.
3,0,واهية للغاية: flimsyif للغاية ، فأنت تشتريه ، كن حذرًا جدًا للغاية مع إطالة وتقصيره ، وأي استخدام على الإطلاق على الإطلاق.(بلا ​​مزاح)
4,1,Pop for Girls and Girly Boys ، والأشخاص الذين يحبون الضحك: عليك فقط أن تبتسم عندما تستمع إلى Book of Love ، وهم فرقة كلها هناك ، وهم ممتعون عندما لا تأخذهم إلى جادة.ليس عليك أن تكون مثليًا للاستمتاع بموسيقى ولكنها ستساعد ، أو على الأقل ودية.شرائه فقط للابتسام ، وشعر بالدوار والغموض في الداخل.


In [ ]:
# Clean the dataset
arabic_pattern = re.compile(r'[^أ-يء \s]+')
def remove_non_arabic(text):
    return arabic_pattern.sub('', text)

df['content'] = df['content'].apply(lambda x: str(x).replace(".", " ").strip())
df['cleaned_content'] = df['content'].apply(remove_non_arabic)
df = df[["cleaned_content", "label"]].dropna()


In [ ]:
df.head()

,cleaned_content,label
0,النعال المريحة أرتدي هذه النعال كثيرافهي دافئة ومريحة وبأسعار معقولة لجودة رائعة زوجي وأنا على حد سواء لدينا زوج ونحن نحبهم,1
1,منتج جميل خدمة سيئة لقد اشتريت زوجا من النعال الباو الدب باتباع إرشادات وصف المنتج ارتفعت حجما للتناسب عندما وصلوا كانوا لطيفين حقا وأحبوا كانت كبيرة جدا حاولت إعادة ترتيب ولم تكن متوفرة حاولت الاتصال بـ عبر الهاتف وقيل لها البريد الإلكتروني لقد أرسلت عبر البريد الإلكتروني معضلي وانتظرت أسبوعا ولم ترد لقد أرسلت رسالتي الإلكترونية مرة أخرى قبل ثلاثة أيام لا يوجد استجابة أريد فقط استبدال العنصر أو الحصول على إجابة,1
2,جيد للأشياء الصغيرة هذا يعمل بشكل جيد لالتقاط قطع صغيرة من المجوهرات ولكن الذهاب ببطء إنه مفيد لكن المغناطيس ليس قويا جدا,1
3,واهية للغاية للغاية فأنت تشتريه كن حذرا جدا للغاية مع إطالة وتقصيره وأي استخدام على الإطلاق على الإطلاق بلا مزاح,0
4,والأشخاص الذين يحبون الضحك عليك فقط أن تبتسم عندما تستمع إلى وهم فرقة كلها هناك وهم ممتعون عندما لا تأخذهم إلى جادة ليس عليك أن تكون مثليا للاستمتاع بموسيقى ولكنها ستساعد أو على الأقل ودية شرائه فقط للابتسام وشعر بالدوار والغموض في الداخل,1


In [ ]:

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['cleaned_content'], df['label'], test_size=0.3, stratify=df['label'], random_state=42
)

print(f"Training Samples: {len(train_texts)}, Testing Samples: {len(test_texts)}")


Training Samples: 231000, Testing Samples: 99000


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/334k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Inputs:

`texts`: A list of text strings to tokenize.
`batch_size`: The number of texts to process at once

**Initialization:**

`all_encodings` is a dictionary to store tokenized outputs (input_ids and attention_mask).

**Batching:**

- For each batch:
- It tokenizes the texts
- Applies padding
- Truncates longer sequences.
- Converts outputs to PyTorch tensors

**Concatenation:**

After processing all batches, it concatenates the `input_ids` and `attention_mask` tensors along the first dimension(rows).

**Output:**

Returns a dictionary with two keys:
- `input_ids`: A tensor of tokenized text IDs.
- `attention_mask`: A tensor indicating which tokens should be attended.

In [ ]:
# Function for batched tokenization
def batch_tokenize(texts, batch_size=1000):
    """Tokenize text data in batches and ensure uniform padding."""
    all_encodings = {"input_ids": [], "attention_mask": []}

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]

        # Tokenize batch with fixed padding to max_length=400
        encoded_batch = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )

        all_encodings["input_ids"].append(encoded_batch["input_ids"])
        all_encodings["attention_mask"].append(encoded_batch["attention_mask"])

    #  Now all tensors have the same size, so concatenation works
    all_encodings["input_ids"] = torch.cat(all_encodings["input_ids"], dim=0)
    all_encodings["attention_mask"] = torch.cat(all_encodings["attention_mask"], dim=0)

    return all_encodings


In [ ]:
# Apply batched tokenization
train_texts_list  = train_texts.dropna().tolist()
test_texts_list = test_texts.dropna().tolist()
encoded_train = batch_tokenize(train_texts_list, batch_size=1000)
encoded_test = batch_tokenize(test_texts_list, batch_size=1000)
print(f"Train Input IDs Shape: {encoded_train['input_ids'].shape}")
print(f"Test Input IDs Shape: {encoded_test['input_ids'].shape}")

Train Input IDs Shape: torch.Size([231000, 128])
Test Input IDs Shape: torch.Size([99000, 128])


In [ ]:
tokens = encoded_train['input_ids'][1]
tokenizer.decode(tokens)

'[CLS] قرص مضغوط ممتاز معظم الناس يعرفون عن لكن من المسلم به انني كنت على دراية به قبل ذلك عبر مجموعات يورو التي تضمنت ضربته انا اماديوس بالنسبة لاوليك الذين ليسوا على دراية بموسيقاه فان الاغاني باللغة النمساوية اي الالمانية القياسية مع بعض الاغاني التي تحتوي على صوت وغيرها من نفس الفترة لهم فقط لاعطايك فكرة عن تنوعه بعد قولي هذا فالكو هو فالكو وبيعه باختصار بالمقارنة مع الخرين غير عادل استمع الى اللدغات الصوتية لاغانيه وستحصل على فكرة عن مدى موهبته [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [ ]:
tokens = encoded_test['input_ids'][1]
tokenizer.decode(tokens)

'[CLS] فيلم رايع ساشتري هذا الفيلم ومشاهدته مرارا وتكرارا مرارا وتكرارا لانني لن اتعب منه ابدا [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [ ]:
sample_index = 1

sample_text = train_texts_list[sample_index]

sample_token_ids = encoded_train["input_ids"][sample_index]

print(f'Text: {sample_text}')
print(f'Number of tokens: {len(sample_token_ids)}')

for token in sample_token_ids:
    print(f'Token ID: {token.item()}, Subword: {tokenizer.decode([token.item()])}')


Text: قرص مضغوط ممتاز معظم الناس يعرفون  عن     لكن من المسلم به أنني كنت على دراية به قبل ذلك عبر مجموعات يورو التي تضمنت ضربته   أنا أماديوس بالنسبة لأولئك الذين ليسوا على دراية بموسيقاه  فإن الأغاني باللغة النمساوية أي الألمانية القياسية مع بعض الأغاني التي تحتوي على صوت   وغيرها من نفس الفترة  لهم فقط لإعطائك فكرة عن تنوعه بعد قولي هذا  فالكو هو فالكو وبيعه باختصار بالمقارنة مع الخرين غير عادل استمع إلى اللدغات الصوتية لأغانيه وستحصل على فكرة عن مدى موهبته
Number of tokens: 128
Token ID: 2, Subword: [CLS]
Token ID: 18695, Subword: قرص
Token ID: 3214, Subword: مض
Token ID: 8194, Subword: ##غوط
Token ID: 11774, Subword: ممتاز
Token ID: 4537, Subword: معظم
Token ID: 2538, Subword: الناس
Token ID: 13649, Subword: يعرفون
Token ID: 1760, Subword: عن
Token ID: 2356, Subword: لكن
Token ID: 1726, Subword: من
Token ID: 5111, Subword: المسلم
Token ID: 2037, Subword: به
Token ID: 7066, Subword: انني
Token ID: 2779, Subword: كنت
Token ID: 1747, Subword: على
Token ID: 27446, Subword: دراية
Token

# get the embedding


`bert_model.eval()`>>disables dropout,Batch Normalization...

In [ ]:
bert_model = AutoModel.from_pretrained("asafaya/bert-base-arabic").to("cuda")
bert_model.eval()  # Set model to evaluation mode

# Function to generate BERT CLS embeddings
def get_bert_embeddings(encoded_inputs, model, batch_size=32):
    """Extract BERT embeddings from pre-tokenized inputs."""
    dataset = torch.utils.data.TensorDataset(encoded_inputs["input_ids"], encoded_inputs["attention_mask"])
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

    all_cls_embeddings = []
    with torch.no_grad():
        for batch in dataloader:
            batch_inputs = {"input_ids": batch[0].to("cuda"), "attention_mask": batch[1].to("cuda")}
            outputs = model(**batch_inputs)
            cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu()  # Extract CLS token embeddings
            all_cls_embeddings.append(cls_embeddings)

    return torch.cat(all_cls_embeddings, dim=0).numpy()  # Convert to NumPy


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
# Generate embeddings for train and test datasets
train_embeddings = get_bert_embeddings(encoded_train, bert_model)
np.save("/kaggle/working/train_embeddings.npy", train_embeddings)

In [ ]:
print(f"Final train CLS Embeddings Shape: {train_embeddings.shape}")

Final train CLS Embeddings Shape: (231000, 768)


In [ ]:
test_embeddings = get_bert_embeddings(encoded_test, bert_model)
np.save("/kaggle/working/test_embeddings.npy", test_embeddings)


In [ ]:
print(f"Final test CLS Embeddings Shape: {test_embeddings.shape}")

Final test CLS Embeddings Shape: (99000, 768)


# adding classification layer

## Reshape embeddings to fit RNN/LSTM input shape
**(batch_size, time_steps=1, features=768)**

In [ ]:
train_embeddings_rnn = np.expand_dims(train_embeddings, axis=1)  # Shape: (231000, 1, 768)
test_embeddings_rnn = np.expand_dims(test_embeddings, axis=1)    # Shape: (99000, 1, 768)

#n_labels = train_labels.to_numpy()
test_labels = test_labels.to_numpy()

print(f"Train Embeddings Shape: {train_embeddings_rnn.shape}")  # (231000, 1, 768)
print(f"Test Embeddings Shape: {test_embeddings_ Convert labels to NumPy arrays (if not already)
trairnn.shape}")

Train Embeddings Shape: (231000, 1, 768)
Test Embeddings Shape: (99000, 1, 768)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout, LSTM, Bidirectional, Input
from tensorflow.keras.callbacks import ModelCheckpoint

## RNN
------------------------------

In [ ]:
# Define RNN model
rnn_model = Sequential([
    SimpleRNN(128, activation='relu', input_shape=(1, 768)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile model
rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
rnn_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn_1 (SimpleRNN)             │ (None, 128)                 │         114,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 123,137 (481.00 KB)

 Trainable params: 123,137 (481.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
checkpoint_filepath = "/kaggle/working/rnn/weights.{epoch:02d}.keras"
modelcheckpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',#on metric decrease
    save_best_only=True
)

In [ ]:
# Train the RNN model
rnn_model.fit(
    train_embeddings_rnn, train_labels,
    epochs=10, batch_size=32,
    validation_data=(test_embeddings_rnn, test_labels),
    callbacks=[modelcheckpoint]
)

Epoch 1/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.7731 - loss: 0.4775 - val_accuracy: 0.8268 - val_loss: 0.3948
Epoch 2/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.8169 - loss: 0.4103 - val_accuracy: 0.8313 - val_loss: 0.3846
Epoch 3/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.8210 - loss: 0.4013 - val_accuracy: 0.8343 - val_loss: 0.3773
Epoch 4/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.8241 - loss: 0.3957 - val_accuracy: 0.8320 - val_loss: 0.3809
Epoch 5/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.8259 - loss: 0.3917 - val_accuracy: 0.8358 - val_loss: 0.3735
Epoch 6/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.8287 - loss: 0.3881 - val_accuracy: 0.8360 - val_loss: 0.3812
Epoch 7/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.8292 - loss: 0.3860 - val_accuracy: 0.8379 - val_loss: 0.3823
Epoch 8/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.8305 - loss: 0

In [ ]:
# Evaluate the RNN model
test_loss, test_acc = rnn_model.evaluate(test_embeddings_rnn, test_labels)
print(f"RNN Test Accuracy: {test_acc * 100:.2f}%")
print(f"RNN Test loss: {test_loss * 100:.2f}%")

3094/3094 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8371 - loss: 0.3797
RNN Test Accuracy: 83.61%
RNN Test loss: 38.16%


## LSTM
----------------------------

In [ ]:
# Define LSTM model
lstm_model = Sequential([
    LSTM(128, activation='relu', input_shape=(1, 768)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

In [ ]:
# Compile model
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
rnn_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn_1 (SimpleRNN)             │ (None, 128)                 │         114,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 369,413 (1.41 MB)

 Trainable params: 123,137 (481.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 246,276 (962.02 KB)

In [ ]:
checkpoint_filepath = "/kaggle/working/lstm/weights.{epoch:02d}.keras"
modelcheckpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',#on metric decrease
    save_best_only=True
)

In [ ]:

# Train the LSTM model
lstm_model.fit(
    train_embeddings_rnn, train_labels,
    epochs=10, batch_size=32,
    validation_data=(test_embeddings_rnn, test_labels),
    callbacks=[modelcheckpoint]
)


Epoch 1/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - accuracy: 0.7821 - loss: 0.4664 - val_accuracy: 0.8250 - val_loss: 0.3964
Epoch 2/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.8199 - loss: 0.4019 - val_accuracy: 0.8322 - val_loss: 0.3764
Epoch 3/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.8253 - loss: 0.3942 - val_accuracy: 0.8368 - val_loss: 0.3749
Epoch 4/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.8273 - loss: 0.3871 - val_accuracy: 0.8344 - val_loss: 0.3790
Epoch 5/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.8317 - loss: 0.3826 - val_accuracy: 0.8390 - val_loss: 0.3740
Epoch 6/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.8327 - loss: 0.3781 - val_accuracy: 0.8395 - val_loss: 0.3658
Epoch 7/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.8318 - loss: 0.3793 - val_accuracy: 0.8410 - val_loss: 0.3600
Epoch 8/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.8343 - loss: 0

## Bi_LSTM
----------------------------

In [ ]:
# Build the Bi-Directional LSTM model using an Input layer
bi_lstm_model = Sequential([
    # Define the input shape explicitly
    Input(shape=(train_embeddings_rnn.shape[1], train_embeddings_rnn.shape[2])),  # e.g., (1, 768)
    Bidirectional(LSTM(128, activation='relu')),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # For binary classification
])

In [ ]:
bi_lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
bi_lstm_model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional)      │ (None, 256)                 │         918,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 64)                  │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 935,041 (3.57 MB)

 Trainable params: 935,041 (3.57 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
bi_lstm_model.fit(
    train_embeddings_rnn, train_labels,
    epochs=10,
    batch_size=32,
    validation_data=(test_embeddings_rnn, test_labels))

Epoch 1/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.7838 - loss: 0.4613 - val_accuracy: 0.8257 - val_loss: 0.3867
Epoch 2/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.8197 - loss: 0.4034 - val_accuracy: 0.8345 - val_loss: 0.3718
Epoch 3/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.8271 - loss: 0.3920 - val_accuracy: 0.8350 - val_loss: 0.3713
Epoch 4/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.8309 - loss: 0.3834 - val_accuracy: 0.8366 - val_loss: 0.3636
Epoch 5/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.8339 - loss: 0.3771 - val_accuracy: 0.8420 - val_loss: 0.3594
Epoch 6/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.8354 - loss: 0.3743 - val_accuracy: 0.8422 - val_loss: 0.3572
Epoch 7/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.8365 - loss: 0.3729 - val_accuracy: 0.8438 - val_loss: 0.3544
Epoch 8/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.8379 - loss: 0

## Bi_RNN
----------------------------

In [ ]:
# Build the Bi-Directional LSTM model using an Input layer
bi_rnn_model = Sequential([
    # Define the input shape explicitly
    Input(shape=(train_embeddings_rnn.shape[1], train_embeddings_rnn.shape[2])),  # e.g., (1, 768)
    Bidirectional(SimpleRNN(128, activation='relu')),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # For binary classification
])

In [ ]:
bi_rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
bi_rnn_model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_4 (Bidirectional)      │ (None, 256)                 │         229,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 64)                  │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 246,145 (961.50 KB)

 Trainable params: 246,145 (961.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
bi_rnn_model.fit(
    train_embeddings_rnn, train_labels,
    epochs=10,
    batch_size=32,
    validation_data=(test_embeddings_rnn, test_labels))

Epoch 1/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.7795 - loss: 0.4704 - val_accuracy: 0.8292 - val_loss: 0.3799
Epoch 2/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.8189 - loss: 0.4045 - val_accuracy: 0.8326 - val_loss: 0.3748
Epoch 3/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.8248 - loss: 0.3932 - val_accuracy: 0.8363 - val_loss: 0.3729
Epoch 4/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.8281 - loss: 0.3885 - val_accuracy: 0.8382 - val_loss: 0.3641
Epoch 5/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.8308 - loss: 0.3831 - val_accuracy: 0.8396 - val_loss: 0.3631
Epoch 6/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.8325 - loss: 0.3789 - val_accuracy: 0.8414 - val_loss: 0.3596
Epoch 7/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.8329 - loss: 0.3778 - val_accuracy: 0.8397 - val_loss: 0.3638
Epoch 8/10
7219/7219 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.8363 - loss: 0